<h1 align="center"> Project Setup </h1>

#### Importing Dependencies

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans

#### Importing Datasets

In [2]:
# Display all columns in the DataFrame using pandas settings
pd.set_option('display.max_columns', None)

In [3]:
df_imports = pd.read_excel('data/WtoData_worldwide_import_from_2010_to_2022_all_countries.xlsx')

<br><br>

<h1 align="center"> Summary</h1>

### Objective

This project focuses on how the country will be able to know the right market to export their products. This will allow the country to get a good guide on foreign trade, and know which countries have High demand in different sectors, especially agricultural products.

The specific objectives of this project are:
1. Analyzethedatasetthatwearegoingtocollecttobetterunderstandandgive
some insight on the international trade
2. ChoosethebestplacewiththeHighdemandforagriculturalproductssothat Haiti can promote and sell its products

<hr>

### Hypothesis: Research Question?

What is the question that you would like to answer in order to make a decision.

<hr>

### Data Source

For this project we will be using the World Trade Organization(WTO) data portal to have access to. The WTO Data portal contains statistical indicators, Available time series cover merchandise trade and trade in services statistics, market access indicators (bound, applied & preferential tariffs), non-tariff information as well as other indicators.

<br><br>

<h1 align="center"> Data Cleaning </h1>

#### Data Overview

In [4]:
# Displaying the countries import product dataframe
print('------ Imports Dataset ------')
display(df_imports.head(4))

------ Imports Dataset ------


,Indicator,Merchandise imports by product group – annual (Million US dollar),Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Reporting Economy,Product/Sector,Partner Economy,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
2,World,SI3_AGG - TO - Total merchandise,World,15438092.0,18438364.0,18657296.0,18966119.0,19060809.0,16733507.0,16211194.0,17985896.0,19836342.0,19284167.0,17812107.0
3,World,SI3_AGG - AG - Agricultural products,World,1391529.0,1701893.0,1681611.0,1756648.0,1798003.0,1594510.0,1599989.0,1759978.0,1851346.0,1823105.0,NaN


In [5]:
print('------------------------------ Dataset Shape ------------------------------')
print('The Imports dataset has',df_imports.shape[0], 'Rows and', df_imports.shape[1],'columns')

print('------------------------------ Dataframe Columns ------------------------------')
display(df_imports.columns)

print('------------------------------ Data types ------------------------------')
display(df_imports.dtypes)

print('------------------------------ dataframe info ------------------------------')
display(df_imports.info())

print('------------------------------ Statistical data ------------------------------')
display(df_imports.describe())

------------------------------ Dataset Shape ------------------------------
The Imports dataset has 3287 Rows and 14 columns
------------------------------ Dataframe Columns ------------------------------


Index(['Indicator',
       '  Merchandise imports by product group – annual (Million US dollar)',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')

------------------------------ Data types ------------------------------


Indicator                                                               object
  Merchandise imports by product group – annual (Million US dollar)     object
Unnamed: 2                                                              object
Unnamed: 3                                                             float64
Unnamed: 4                                                             float64
Unnamed: 5                                                             float64
Unnamed: 6                                                             float64
Unnamed: 7                                                             float64
Unnamed: 8                                                             float64
Unnamed: 9                                                             float64
Unnamed: 10                                                            float64
Unnamed: 11                                                            float64
Unnamed: 12                                         

------------------------------ dataframe info ------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 14 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Indicator                                                            3286 non-null   object 
 1     Merchandise imports by product group – annual (Million US dollar)  3286 non-null   object 
 2   Unnamed: 2                                                           3286 non-null   object 
 3   Unnamed: 3                                                           3140 non-null   float64
 4   Unnamed: 4                                                           3177 non-null   float64
 5   Unnamed: 5                                                           3131 non-null   float64
 6   Unnamed: 6                 

None

------------------------------ Statistical data ------------------------------


,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
count,3.140000e+03,3.177000e+03,3.131000e+03,3.111000e+03,3.100000e+03,3.116000e+03,3.101000e+03,3.135000e+03,3.136000e+03,3.136000e+03,2.080000e+02
mean,3.143869e+04,3.675237e+04,3.758531e+04,3.860113e+04,3.913586e+04,3.433887e+04,3.362719e+04,3.711064e+04,4.083153e+04,3.967764e+04,1.818519e+05
std,3.675592e+05,4.329523e+05,4.398730e+05,4.502819e+05,4.557177e+05,4.060061e+05,3.974724e+05,4.376012e+05,4.794420e+05,4.672140e+05,1.262847e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.127500e+02,1.210000e+02,1.360000e+02,1.440000e+02,1.600000e+02,1.437500e+02,1.540000e+02,1.710000e+02,1.830000e+02,1.820000e+02,1.714250e+03
50%,8.240000e+02,9.460000e+02,1.000000e+03,1.061000e+03,1.133000e+03,1.032500e+03,1.003000e+03,1.095000e+03,1.213500e+03,1.206000e+03,7.946000e+03
75%,5.285750e+03,6.199000e+03,6.518000e+03,6.834500e+03,7.124500e+03,6.023250e+03,5.817000e+03,6.286500e+03,6.971000e+03,6.792750e+03,4.433500e+04
max,1.543809e+07,1.843836e+07,1.865730e+07,1.896612e+07,1.906081e+07,1.673351e+07,1.621119e+07,1.798590e+07,1.983634e+07,1.928417e+07,1.781211e+07


In [6]:
# Display all columns
print('-------- Imports column names --------')
display(df_imports.columns)

# Change columns name
map_cols_name = {
    'Indicator':'Reporting Economy',
    '  Merchandise imports by product group – annual (Million US dollar)': 'Product/Sector',
    'Unnamed: 2': 'Partner Economy',
    'Unnamed: 3': '2010',
    'Unnamed: 4': '2011',
    'Unnamed: 5': '2012',
    'Unnamed: 6': '2013',
    'Unnamed: 7': '2014',
    'Unnamed: 8': '2015',
    'Unnamed: 9': '2016',
    'Unnamed: 10': '2017',
    'Unnamed: 11': '2018',
    'Unnamed: 12': '2019',
    'Unnamed: 13': '2020',
}

# Change all default column names
renamed_cols_df = df_imports.rename(columns=map_cols_name)

print('-------- Imports Dataframe with new cols names --------')
display(renamed_cols_df.head(5))

-------- Imports column names --------


Index(['Indicator',
       '  Merchandise imports by product group – annual (Million US dollar)',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')

-------- Imports Dataframe with new cols names --------


,Reporting Economy,Product/Sector,Partner Economy,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Reporting Economy,Product/Sector,Partner Economy,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
2,World,SI3_AGG - TO - Total merchandise,World,15438092.0,18438364.0,18657296.0,18966119.0,19060809.0,16733507.0,16211194.0,17985896.0,19836342.0,19284167.0,17812107.0
3,World,SI3_AGG - AG - Agricultural products,World,1391529.0,1701893.0,1681611.0,1756648.0,1798003.0,1594510.0,1599989.0,1759978.0,1851346.0,1823105.0,NaN
4,World,SI3_AGG - AGFO - Food,World,1144082.0,1387912.0,1391766.0,1465158.0,1510854.0,1340065.0,1354624.0,1482990.0,1551785.0,1547817.0,NaN


In [7]:
# Remove the first 2 rows in the imports dataframe
to_drop_rows = renamed_cols_df.index[:2]

dropped_rows_df = renamed_cols_df.drop(to_drop_rows).reset_index(drop=True)

display(dropped_rows_df.head())

,Reporting Economy,Product/Sector,Partner Economy,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,World,SI3_AGG - TO - Total merchandise,World,15438092.0,18438364.0,18657296.0,18966119.0,19060809.0,16733507.0,16211194.0,17985896.0,19836342.0,19284167.0,17812107.0
1,World,SI3_AGG - AG - Agricultural products,World,1391529.0,1701893.0,1681611.0,1756648.0,1798003.0,1594510.0,1599989.0,1759978.0,1851346.0,1823105.0,NaN
2,World,SI3_AGG - AGFO - Food,World,1144082.0,1387912.0,1391766.0,1465158.0,1510854.0,1340065.0,1354624.0,1482990.0,1551785.0,1547817.0,NaN
3,World,SI3_AGG - MI - Fuels and mining products,World,3183852.0,4278365.0,4227403.0,4208130.0,3874299.0,2489603.0,2124334.0,2755203.0,3408176.0,3168025.0,NaN
4,World,SI3_AGG - MIFU - Fuels,World,2462969.0,3361172.0,3401742.0,3403308.0,3091267.0,1859835.0,1528358.0,2006039.0,2570707.0,2350244.0,NaN


#### Treatment of Missing Values

In [8]:
# Checking for missing values in the import dataset
dropped_rows_df.isnull().sum()

Reporting Economy       0
Product/Sector          0
Partner Economy         0
2010                  146
2011                  109
2012                  155
2013                  175
2014                  186
2015                  170
2016                  185
2017                  151
2018                  150
2019                  150
2020                 3078
dtype: int64

In [9]:
# check if we don't have Duplicated values in the dataframe
print(dropped_rows_df.duplicated().sum(), 'value')

0 value


In [10]:
# Dealing with missing values with the fillna function
dropped_rows_df = dropped_rows_df.fillna(0)

In [11]:
print('---- World import data ----')
display(dropped_rows_df.isna().sum())


print('---------------')
display('Now all of our empty values have been successfully filled with 0')

---- World import data ----


Reporting Economy    0
Product/Sector       0
Partner Economy      0
2010                 0
2011                 0
2012                 0
2013                 0
2014                 0
2015                 0
2016                 0
2017                 0
2018                 0
2019                 0
2020                 0
dtype: int64

---------------


'Now all of our empty values have been successfully filled with 0'

#### Final Dataframe

In [12]:
# Change date type on both dataset
print('-------- import old data type --------')
display(dropped_rows_df.dtypes)


# Change Float to Integer on both datasets
cols = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']

dropped_rows_df[cols] = dropped_rows_df[cols].apply(np.int32)


print('-------- New import data type --------')
display(dropped_rows_df.dtypes)

-------- import old data type --------


Reporting Economy     object
Product/Sector        object
Partner Economy       object
2010                 float64
2011                 float64
2012                 float64
2013                 float64
2014                 float64
2015                 float64
2016                 float64
2017                 float64
2018                 float64
2019                 float64
2020                 float64
dtype: object

-------- New import data type --------


Reporting Economy    object
Product/Sector       object
Partner Economy      object
2010                  int32
2011                  int32
2012                  int32
2013                  int32
2014                  int32
2015                  int32
2016                  int32
2017                  int32
2018                  int32
2019                  int32
2020                  int32
dtype: object

In [13]:
print('------------ Display values in the Product/Sectors on the Import dataset columns ------------')
display(dropped_rows_df['Product/Sector'].value_counts())

# Remove all non use coverage CODE before the last hyphen in all value in the "Product/sector" column
dropped_rows_df['Product/Sector'] = dropped_rows_df['Product/Sector'].apply(lambda x: x.split('- ')[-1])

# Also drop the ['Partner Economy']columns
df = dropped_rows_df.drop(['Partner Economy'],axis=1)

print('---- Final import dataframe ----')
display(df.head())

------------ Display values in the Product/Sectors on the Import dataset columns ------------


SI3_AGG - TO - Total merchandise                                        207
SI3_AGG - MA - Manufactures                                             182
SI3_AGG - MACH - Chemicals                                              182
SI3_AGG - AG - Agricultural products                                    182
SI3_AGG - MAMT - Machinery and transport equipment                      182
SI3_AGG - MIFU - Fuels                                                  182
SI3_AGG - AGFO - Food                                                   182
SI3_AGG - MI - Fuels and mining products                                182
SI3_AGG - MAIS - Iron and steel                                         181
SI3_AGG - MAMTAU - Automotive products                                  181
SI3_AGG - MAMTTE - Transport equipment                                  181
SI3_AGG - MAMTOTTL - Telecommunications equipment                       181
SI3_AGG - MACL - Clothing                                               181
SI3_AGG - MA

---- Final import dataframe ----


,Reporting Economy,Product/Sector,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,World,Total merchandise,15438092,18438364,18657296,18966119,19060809,16733507,16211194,17985896,19836342,19284167,17812107
1,World,Agricultural products,1391529,1701893,1681611,1756648,1798003,1594510,1599989,1759978,1851346,1823105,0
2,World,Food,1144082,1387912,1391766,1465158,1510854,1340065,1354624,1482990,1551785,1547817,0
3,World,Fuels and mining products,3183852,4278365,4227403,4208130,3874299,2489603,2124334,2755203,3408176,3168025,0
4,World,Fuels,2462969,3361172,3401742,3403308,3091267,1859835,1528358,2006039,2570707,2350244,0


<br><br>

<h1 align="center"> Analytical Transformations </h1>

In [14]:
# Perform any transformation on the columns in the dataset to enable further analysis.
df_melt = df.copy()

In [32]:
# Melt the Dataframe
df_melt = pd.melt(frame=df, 
                  id_vars=['Product/Sector','Reporting Economy'], 
                  var_name='Year', 
                  value_name="Million US dollar")

# Reshape the dataframe using pivot_table
reshape = df_melt.pivot_table(columns='Product/Sector',
                             index=['Year','Reporting Economy'],
                             values="Million US dollar").fillna(0)

# Convert the new dataframe to int
reshape = reshape.astype(int)

# Reset the index to have a beautifull dataframe
reshape = reshape.reset_index()

# Remove Index name
reshape = reshape.rename_axis(None, axis=1)

# Change columns position
print('------------------ Display all columns ------------------')
display(reshape.columns)
print('------------------ Swap columns ------------------')
reshape = reshape[['Year','Reporting Economy','Agricultural products',
                   'Automotive products','Chemicals','Clothing',
                   'Electronic data processing and office equipment','Food','Fuels',
                   'Fuels and mining products','Integrated circuits and electronic components',
                  'Machinery and transport equipment','Manufactures','Office and telecom equipment',
                   'Pharmaceuticals','Telecommunications equipment','Textiles',
                   'Transport equipment','Total merchandise']]

# Overview our dataframe
reshape.head()

------------------ Display all columns ------------------


Index(['Year', 'Reporting Economy', 'Agricultural products',
       'Automotive products', 'Chemicals', 'Clothing',
       'Electronic data processing and office equipment', 'Food', 'Fuels',
       'Fuels and mining products',
       'Integrated circuits and electronic components', 'Iron and steel',
       'Machinery and transport equipment', 'Manufactures',
       'Office and telecom equipment', 'Pharmaceuticals',
       'Telecommunications equipment', 'Textiles', 'Total merchandise',
       'Transport equipment'],
      dtype='object')

------------------ Swap columns ------------------


,Year,Reporting Economy,Agricultural products,Automotive products,Chemicals,Clothing,Electronic data processing and office equipment,Food,Fuels,Fuels and mining products,Integrated circuits and electronic components,Machinery and transport equipment,Manufactures,Office and telecom equipment,Pharmaceuticals,Telecommunications equipment,Textiles,Transport equipment,Total merchandise
0,2010,Afghanistan,706,193,82,12,0,706,1075,1090,0,339,984,19,0,19,118,197,5154
1,2010,Albania,872,217,464,173,63,826,635,802,14,875,2731,176,155,99,168,241,4406
2,2010,Algeria,7350,3981,4452,183,520,6683,867,1493,96,16716,31367,1215,1719,599,351,5108,40473
3,2010,Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2010,Angola,2882,1204,963,127,174,2764,3105,3233,19,6475,10521,561,148,369,108,2090,16667


# <h1 align="center"> Data Analysis </h1>

#### Descriptive Statistical Analysis

In [17]:
# Using basic statistical measures such as measurements of central tendancy such as mean, median and mode.

In [18]:
# Find the average for each numeric columns
df.mean()

2010    30050.376865
2011    35543.459665
2012    35822.707763
2013    36555.889498
2014    36931.251142
2015    32571.660883
2016    31743.041400
2017    35415.471842
2018    38978.888280
2019    37877.336986
2020    11513.902283
dtype: float64

In [19]:
# Describe
df.describe()

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,3.285000e+03,3.285000e+03,3.285000e+03,3.285000e+03,3.285000e+03,3.285000e+03,3.285000e+03,3.285000e+03,3.285000e+03,3.285000e+03,3.285000e+03
mean,3.005038e+04,3.554346e+04,3.582271e+04,3.655589e+04,3.693125e+04,3.257166e+04,3.174304e+04,3.541547e+04,3.897889e+04,3.787734e+04,1.151390e+04
std,3.594112e+05,4.258241e+05,4.295090e+05,4.382760e+05,4.427875e+05,3.954940e+05,3.862543e+05,4.275607e+05,4.685164e+05,4.565666e+05,3.201338e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.100000e+01,9.700000e+01,8.900000e+01,9.300000e+01,1.020000e+02,9.500000e+01,9.700000e+01,1.190000e+02,1.320000e+02,1.310000e+02,0.000000e+00
50%,7.060000e+02,8.310000e+02,8.580000e+02,8.950000e+02,9.310000e+02,8.360000e+02,7.920000e+02,9.170000e+02,1.017000e+03,1.012000e+03,0.000000e+00
75%,4.767000e+03,5.843000e+03,5.916000e+03,6.180000e+03,6.390000e+03,5.450000e+03,5.169000e+03,5.734000e+03,6.391000e+03,6.200000e+03,0.000000e+00
max,1.543809e+07,1.843836e+07,1.865730e+07,1.896612e+07,1.906081e+07,1.673351e+07,1.621119e+07,1.798590e+07,1.983634e+07,1.928417e+07,1.781211e+07


In [20]:
df.corr()

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
2010,1.000000,0.999318,0.999233,0.999390,0.999718,0.997294,0.995480,0.997250,0.998528,0.997983,0.767880
2011,0.999318,1.000000,0.999880,0.999788,0.999296,0.994418,0.991796,0.994276,0.996359,0.995507,0.773789
2012,0.999233,0.999880,1.000000,0.999921,0.999418,0.994635,0.991981,0.994430,0.996500,0.995657,0.776743
2013,0.999390,0.999788,0.999921,1.000000,0.999679,0.995425,0.992953,0.995270,0.997176,0.996402,0.774066
2014,0.999718,0.999296,0.999418,0.999679,1.000000,0.997465,0.995556,0.997320,0.998648,0.998121,0.770218
2015,0.997294,0.994418,0.994635,0.995425,0.997465,1.000000,0.999696,0.999863,0.999531,0.999689,0.757708
2016,0.995480,0.991796,0.991981,0.992953,0.995556,0.999696,1.000000,0.999666,0.998769,0.999159,0.751489
2017,0.997250,0.994276,0.994430,0.995270,0.997320,0.999863,0.999666,1.000000,0.999674,0.999836,0.753128
2018,0.998528,0.996359,0.996500,0.997176,0.998648,0.999531,0.998769,0.999674,1.000000,0.999927,0.758077
2019,0.997983,0.995507,0.995657,0.996402,0.998121,0.999689,0.999159,0.999836,0.999927,1.000000,0.756172


In [21]:
# Make a pairwise scatterplot

#### Distribution of Variables

In [22]:
# Identify the distribution of the data to understand the range of values and how the data is structured.

#### Outliers in the dataset

In [23]:
# Identify if there are any outliers in the dataset based on statistical measures.

<br><br>

<h1 align="center"> Reflections </h1>

#### Summary of Data Analysis

In [24]:
# What insights should the user takeaway from EDA.

#### Questions unanswered

In [25]:
# What aspects of the research question were we unable to answer and why?

#### Recommendations

In [26]:
# Identify if there are any outliers in the dataset based on statistical measures.

#### Next Steps

In [27]:
# What will the analyst do next based on the analysis?